This tutorial is part of the lecture: "Machine Learning in Electrical Engineering and Information Technology" at Hamburg University of Technology (TUHH).

Copyright: 
Dr.-Ing. Fabian Lurz, 
Institute of High-Frequency Technology,
21073 Hamburg, Germany,
fabian.lurz@ieee.org,
May, 2023

**The dataset and code is provided by the Institute of High-Frequency Technology (IHF) of the Hamburg University of Technology exclusively for use in the course "Machine Learning in Electrical Engineering and Information Technology". Please note that the use of the dataset and/or code for other purposes is not permitted.**

# Tutorial 3: Building and Training Artificial Neural Networks for Radar-Based Gesture Recognition

## Introduction 

Welcome to this interactive session in which we will train artificial neural networks for radar-based hand gesture recognition. 

The gestures were recorded with an Infineon BGT60TR13C 60 GHz FMCW radar system. The system was lying on the table and looking upwards.
<img src="img/recording_setup.png" alt="Recording setup" style="width: 250px;"/>

Since this tutorial focuses on machine learning rather than radar signal processing, we use already preprocessed spectrograms of the gestures. In the example below the x-axis represents the time
dimension and the y-axis the physical size which is stated above the respective spectrograms for the gesture 'circle clockwise'. 
<img src="img/exemplary_spectrogram_circle_clockwise.png" alt="Exemplary spectrogram" style="width: 500px;"/>

**Introduction:**
- You should modify the marked parts of the program and you can execute the function section directly. To do this, add the code between ### start ### and ### end ###.
- To execute a function section, you should select it and then press `STRG` + `ENTER`.
- Python 3 is used for this course

## Import of relevant Libraries

At the beginning of each project the most important packages are imported, which are needed in the further course. This includes [Matplotlib](https://matplotlib.org) to generate graphics, [Numpy](https://numpy.org) to perform vector calculations and of course the [Keras](https://keras.io) package from [TensorFlow](https://www.tensorflow.org) to create the neural networks.

**Task:** Run the cell.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split

import torch
from torch import dropout, nn
import torchsummary

## Import the dataset

The dataset consists of the preprocessed gestures `samples.npy` and the corresponding labels `labels.npy`. It is stored in the folder `./gestures_preprocessed/` and can be loaded with the function np.load(file).


**Task:** Import the dataset. <br>
*Note:* The dataset is approx. 60 MB

In [ ]:
### start ####
samples = 
labels = 
### end ###


When loading the dataset, two NumPy arrays are returned.</br>
* The variable `samples` contains contains the pre-processed radar data of the gestures.
* The varialbe `labels` consists of numeric digits representing the 10 different gestures

## Explore the data

Let us first take a closer look at the data set together.
* With `type(variable)` the type of the variable can be displayed. 
* and `len(variable)` gives the number of entries

**Task**: Test both commands to determine the type and length of *samples*, *labels*, and *start_end_idxs*. Furthermore, check if the number of labels corresponds to the number of data points. (Reminder: with the `print(...)` command multiple outputs of a cell can be printed)

In [ ]:
### start ####
...
...
### end ###


The Numpy library has implemented the `variable.shape` attribute, which gives more precise information about the dimensions of the vectors.<br>
**Task:** Use the `shape` attribute for both variables.

In [ ]:
### start ####
dim_samples = ...
dim_labels = ...

print(...)
### end ###



As you can easily see from the "**Dimensions labels: (2500,)**", the dataset contains 2500 gestures. 

To understand how exactly these are stored we need to take a closer look at the "**Dimensions samples: (2500, 4, 60, 32)**"
* `2500`: The number of gestures in the dataset
* `4`: We are using already pre-processed radar signals that are: Range, Doppler, Azimuth and Elevation spectrograms (2D-images)
* `60`: The width of each image: Time axis. Each gesture is recorded with 60 frames. As the frame repetition time is 30 ms, the duration of each gesture is max. 1.8 seconds.
* `32` The height of each image: Range / Doppler / Azimuth / Elevation axis

So for each of our 2500 gestures, we have 4 pictures with 60*32 pixels. In each image 60 represents the time axis and 32 the Range / Doppler / Azimuth / Elevation axis.

## Number of the individual gestures

Furthermore, it is interesting to see how many gestures of each number are present in the data set. For this purpose, the frequency of the respective classes is shown below:

In [ ]:
Number, frequency_in_dataset = np.unique(labels, return_counts=True)
plt.bar(Number, frequency_in_dataset)
plt.xlabel('Number in the image')
plt.ylabel('Quantity in the data set')

As you can see, there are 10 different types of gesture. All gestures are present with the same amount. This is ideal for our use and does not require any further adjustments. Stronger deviations should otherwise be considered here. 

## Gesture set

The different gestures are:
* `left_right`: hand swipe from the left to the right side
* `right_left`: hand swipe from the right to the left side
* `top_down`: hand swipe from the top to the bottom
* `down_top`: hand swipe from the bottom to the top
* `circle_clockwise`: periodic circular hand movement in clockwise direction
* `circle_anticlockwise`: periodic circular hand movement in anticlockwise direction
* `forward`: hand swipe away from the body
* `backward`: hand swipe towards the body
* `finger_wave`: wave single fingers
* `finger_rub`: thumb sliding over fingers

<img src="img/gestures.png" alt="Gestures" style="width: 600px;"/>


In [ ]:
gesture_set = ["left_right", "right_left", "top_down", "down_top", "circle_clockwise", "circle_anticlockwise", "forward", "backward", "finger_wave", "finger_rub"]

## Illustrate the individual gestures

To get a better understanding of the dataset we will plot some gestures. To do so try the Matplotlib function `plt.imshow(dataset)`. 

**Hints:**
* To get a single number from the data set, it can be selected by the "slicing" with `dataset[index]`.
* Slicing also works multi-dimensionally. For example, with `dataset[gesture_index,:,:]` you can retrieve all data for a single gesture selected with gesture_index.
* To simply swap x- and y- axis numPy provides the function [numpy.transpose()](https://numpy.org/doc/stable/reference/generated/numpy.transpose.html) can be used
* To add a caption to the plot you can use plt.title('text')
* To draw multiple plots at once [plt.subplot()](https://www.w3schools.com/python/matplotlib_subplot.asp) can be used
* Remember: The dimension of samples are: (2500, 4, 60, 32)

**Task**: Plot some numbers for the `samples`. Modify gesture_index to display all eight different gestures one after the other and check them for plausibility 

To check your output: gesture_index = 1211 should produce the exact same diagrams as shown in the introduction. 

In [ ]:
gesture_index = 1  # The gesture to plot, modify this value to set through the array 
## start 

# plot Range, Doppler, Azimuth, Elevation for the given gesture_index

# print() the label of the given gesture_index

## stop



## Preparation of the data

### Normalization

Typically, the weights between the neurons of a neural network are initiated in a range between zero and one.  
Now, if values between, e.g., 0-255 (from our dataset so far) are given to the ANN as input, it must adjust to the range in the first steps of training. While this is possible, it unnecessarily increases training time and may make the neural network less reliable.

**Task** Check if the data in our dataset needs to be normalized, if so perform the normalization, or if they are already between 0 - 1.

**Hint** There are existing numpy function which output min and max value of an array

In [ ]:
# Normalization of the data values
### start ###
samples_min = ...
samples_max = ...
### end ###

if samples_min < 0 or samples_max > 1:
    print("Needs normalization!")
else:
    print("No normalization needed.")

## Splitting into test and training data

Finally, it is still necessary to divide the dataset into a test and an evaluation dataset. Our algorithm is trained on the training dataset. Then, the evaluation dataset is predicted with the adapted neural network to evaluate the algorithm.

For this the already existing function `train_test_split` of `Scikit-Learn` is first imported and executed.<br>
The data set and the labels are specified as arguments of the function. Afterwards it is defined how many percent of the data are selected pseudo-randomly into the test data set. Furthermore, it is possible to define by specifying a `random_state` that the same numbers are selected each time this function is executed.

In [ ]:
# Set the desired split ratio
train_ratio = 0.8

# Reshape the samples to (2500, 4*60*32)
reshaped_samples = samples.reshape(samples.shape[0], -1)

# Split the reshaped samples into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(reshaped_samples, labels, train_size=train_ratio, random_state=42)

# Reshape train_samples and test_samples back to their original shape
train_data = train_data.reshape(train_data.shape[0], 4, 60, 32)
test_data = test_data.reshape(test_data.shape[0], 4, 60, 32)


## Building the model

### Network architecture

The spectrogram architecture is a three layered convolutional neural network (CNN) using rectified linear unit (ReLU) activation, batch normalization and max pooling followed by two fully connected layers with, at the end, 10 output neurons for the 10 gesture classes.

In [ ]:
# Define the model
model = nn.Sequential(
    nn.Conv2d(4, 32, 5),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 32, 3),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Dropout(0.3),
    nn.Linear(640, 64),
    nn.BatchNorm1d(64),
    nn.Linear(64, 10),
)

# Check / Display the model 
torchsummary.summary(model, (4, 60, 32))

In [ ]:
# Define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.005)

# Define the loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
# Set the number of epochs and batch size
num_epochs = 50
batch_size = 64

In [ ]:
# Convert train_data and test_data to PyTorch tensors and set the data type
train_data_t = torch.from_numpy(train_data).float()
test_data_t = torch.from_numpy(test_data).float()

# Convert train_labels and test_labels to PyTorch tensors and set the data type
train_labels_t = torch.from_numpy(train_labels)
test_labels_t = torch.from_numpy(test_labels)

## Training loop

In [ ]:
# Lists to store the train and test losses and accuracies
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    # Shuffle and divide the training data into batches
    permutation = torch.randperm(train_data_t.size(0))
    shuffled_data = train_data_t[permutation]
    shuffled_labels = train_labels_t[permutation]
    num_batches = len(train_data_t) // batch_size
    
    for i in range(num_batches):
        # Extract the current batch
        batch_data = shuffled_data[i * batch_size : (i + 1) * batch_size]
        batch_labels = shuffled_labels[i * batch_size : (i + 1) * batch_size]
        
        # Convert batch_labels to torch.long data type
        batch_labels = batch_labels.long()
        
        # Forward propagation
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        
        # Backward propagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Calculate training accuracy
        _, predicted_train = torch.max(outputs.data, 1)
        total_train += batch_labels.size(0)
        correct_train += (predicted_train == batch_labels).sum().item()
    
    # Calculate average training loss and accuracy
    epoch_loss = running_loss / num_batches
    train_accuracy = correct_train / total_train * 100
    
    # Evaluation on the test set
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        test_outputs = model(test_data_t)
        _, predicted_test = torch.max(test_outputs, 1)
        correct_test = (predicted_test == test_labels_t).sum().item()
        test_accuracy = correct_test / len(test_labels_t) * 100
    
    # Store loss and accuracy values
    train_losses.append(epoch_loss)
    test_losses.append(loss.item())
    train_accuracies.append(train_accuracy)
    test_accuracies.append(test_accuracy)
    
    # Print training and test information
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Test Loss: {test_losses[-1]:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

When the training starts, the current values for the *loss* and *accuracy* are displayed for the test and training set respectively. So it is easy to follow the progress of the training already during the training and to notice possible errors early. 

## Evaluation of the training

### Train and Test losses

Now we will display the training history. For this purpose we have written the training progress in the variables `train_losses`, `test_losses`, `train_accuracies`, and `test_accuracies` in the previous step.

In [ ]:
# Plot train and test losses
plt.figure()
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train and Test Loss')
plt.legend()
plt.show()

# Plot train and test accuracies
plt.figure()
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy')
plt.legend()
plt.show()

### Evaluation on the test data

Now we can use the network to predict the part of the data that was split off for evaluation. This is made possible with the attribute `prediction = model.predict(X)` of the class `model`. As a result we get the output of the Softmax layer. An example is shown with the code of the cell under the task. 

**Task:** Check the trained network with the `X_evaluation` dataset and change the input_index to test different results. 

**Hint:** To easily find misclassified samples, check out the next cell

In [ ]:
# Specify the index of the input to predict
input_index = 1

# Convert the input data to a tensor, remove extra dimensions
input_tensor = torch.squeeze(test_data_t[input_index])

# Perform the prediction
with torch.no_grad():
    output = model(input_tensor.unsqueeze(0))
    _, predicted_label = torch.max(output, 1)

# Convert the predicted label to a scalar value
predicted_label = predicted_label.item()


# Plot the gesture 
# Create a figure with 2x2 subplots
fig, axs = plt.subplots(2, 2)
fig.suptitle(f"Predicted Label: {gesture_set[int(predicted_label)]}, \n True Label: {gesture_set[int(test_labels[int(input_index)])]}")  # Add an overall title to the figure

# Subplot 1: Range over time
axs[0, 0].imshow(test_data[input_index, 0, :, :].transpose())
axs[0, 0].set_title('Range over time')

# Subplot 2: Doppler over time
axs[0, 1].imshow(test_data[input_index, 1, :, :].transpose())
axs[0, 1].set_title('Doppler over time')

# Subplot 3: Azimuth over time
axs[1, 0].imshow(test_data[input_index, 2, :, :].transpose())
axs[1, 0].set_title('Azimuth over time')

# Subplot 4: Elevation over time
axs[1, 1].imshow(test_data[input_index, 3, :, :].transpose())
axs[1, 1].set_title('Elevation over time')

# Adjust spacing between subplots
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
wrong_indices = []

# Iterate over the test dataset
for i in range(len(test_data)):
    input_tensor = torch.squeeze(test_data_t[i])
    
    # Perform the prediction
    with torch.no_grad():
        output = model(input_tensor.unsqueeze(0))
        _, predicted_label = torch.max(output, 1)
    
    # Convert the predicted label to a scalar value
    predicted_label = predicted_label.item()
    
    # Get the true label
    true_label = test_labels[i]
    
    # Check if the prediction is correct
    if predicted_label != true_label:
        wrong_indices.append(i)

# Print the indices of misclassified classified samples
print("Indices of the misclassified samples:", wrong_indices)

### Confusions Matrix

Scikit-Learn offers the possibility to generate a Confusions Matrix. In this matrix the predictions are compared with the true values. On the diagonal is the accuracy with which the respective number is predicted. Next to the diagonal is the percentage of incorrectly predicted values.

In [ ]:
# After training and testing, get the predictions on the test set
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    test_outputs = model(test_data_t)
    _, test_predictions_tensor = torch.max(test_outputs.data, 1)
    correct_test = (predicted_test == test_labels_t).sum().item()
    test_accuracy = correct_test / len(test_labels_t) * 100

# Print the accuracy of the final test results
print(f"Final Test Accuracy: {test_accuracy:.2f}%")    
    
# Convert the tensor predictions to a numpy array
test_predictions_numpy = test_predictions_tensor.numpy()

# Calculate the confusion matrix
confusion_mat = confusion_matrix(test_labels, test_predictions_numpy, normalize="true")

# Get the number of classes
num_classes = len(gesture_set)

# Plot the confusion matrix with normalized numbers
plt.figure(figsize=(8, 6))
plt.imshow(confusion_mat, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Normalized Confusion Matrix')
plt.xticks(np.arange(len(gesture_set)), gesture_set, rotation=45, ha='right')
plt.yticks(np.arange(len(gesture_set)), gesture_set)

# Add text annotations of normalized numbers
thresh = confusion_mat.max() / 2.0
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, f'{confusion_mat[i, j]:.2f}', ha='center', va='center',
                 color='white' if confusion_mat[i, j] > thresh else 'black')

plt.tight_layout()
plt.show()

## Optimization and further tests

**Congratulations**, <br>
The foundation stone has been laid and you have trained an artificial neural network that is capable of recognizing hand gestures using a highly-integrated 60 GHz FMCW radar sensor. <br><br>
**However,** the system is not perfect yet and you can start now to optimize the parameters to further improve the prediction. 

**Task:** You can also run some further tests to check the quality of the network and the importance of the data. This can also help you understanding what the network learns from the data. As ideas:
* If you delete the azimuth and elevation angle `samples[:,2,:,:] = 0` and/or `samples[:,3,:,:] = 0` in the samples. What do you expect? Try it out and verify your considerations. 
* If you delete the azimuth and elevation angle `test_data[:,2,:,:] = 0` and/or `test_data[:,3,:,:] = 0` only in the test data. What do you expect? Try it out and verify your considerations. 

* Have you wondered why the **validation loss** is sometimes < than the **training loss**? Or why the test accuracy is sometimes *better* that the train accuracy? If yes, you might find [this explanation (https://twitter.com/aureliengeron/status/1110839223878184960)](https://twitter.com/aureliengeron/status/1110839223878184960) very helpful. <br>
Go back and test what happens to the model when you remove (comment out) the Droput layer. Write down the maximum test accuracy beforehand and compare them. What do you notice?
